In [5]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

import gsm
import utils

plt.ioff()

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=12., azim=-63)

fig2 = plt.figure()
ax2 = fig2.add_subplot()

ax.set_xlim(-0.6, 0.6, auto=False)
ax.set_ylim(-0.6, 0.6, auto=False)
ax.set_zlim(-0.9, -0.2, auto=False)
ax.get_xaxis().set_ticklabels([])
ax.get_yaxis().set_ticklabels([])
ax.get_zaxis().set_ticklabels([])
ax.set_aspect("equal", "box")
# ax.axis("off")

cached_sail_surface = None
cached_sunlight_vector = None
cached_force_vector = None
cached_force_lines = None

def update_sail(
    sunlight_azimuth,
    sunlight_elevation,
    reflectivity,
    specularity,
    front_lambertian,
    back_lambertian,
    front_emissivity,
    back_emissivity,
    relative_billow,
    relative_tip_displacement,
):
    global cached_sail_surface, cached_sunlight_vector, cached_force_vector, cached_force_lines

    boom_half_length = 1
    billow = relative_billow*boom_half_length
    tip_displacement = relative_tip_displacement*boom_half_length

    vecs = gsm.get_cached_square_sail_vectors(boom_half_length, billow, -tip_displacement, 20, True)
    if vecs is None:
        vecs = gsm.get_square_sail_vectors(boom_half_length, billow, -tip_displacement, 20, True)
        if cached_sail_surface is not None:
            cached_sail_surface.remove()
        cached_sail_surface = utils.plot_sail(ax, vecs)

    sunlight_elevation = np.deg2rad(sunlight_elevation)
    sunlight_azimuth = np.deg2rad(sunlight_azimuth)
    sunlight_direction = spher2cart(sunlight_elevation, sunlight_azimuth)

    if cached_sunlight_vector is not None:
        cached_sunlight_vector.remove()
    cached_sunlight_vector = utils.plot_vector(ax, sunlight_direction, label="Sunlight", color=utils.YELLOW)

    J1_geo, J2_geo, J3_geo = gsm.get_geometrical_Js_from_square_sail_vectors(vecs)
    J1, J2, J3 = gsm.apply_coefficients(
        J1_geo,
        J2_geo,
        J3_geo,
        reflectivity,
        specularity,
        front_lambertian,
        back_lambertian,
        front_emissivity,
        back_emissivity,
    )
    force = -gsm.get_force(J1, J2, J3, sunlight_direction)/4

    if cached_force_vector is not None:
        cached_force_vector.remove()
    cached_force_vector = utils.plot_vector(ax, force, label="Total Force", color=utils.BLACK)

    ax.legend(loc="upper right")

    elevation = np.linspace(0, np.pi/2, 20)
    force_mag = np.zeros_like(elevation)
    force_normal_mag = np.zeros_like(elevation)
    force_tangent_mag = np.zeros_like(elevation)
    for i, sunlight_elevation_i in enumerate(elevation):
        sunlight_direction = spher2cart(sunlight_elevation_i, 0)
        force = -gsm.get_force(J1, J2, J3, sunlight_direction)/4
        force_mag[i] = np.linalg.norm(force)
        force_normal_vec = np.dot( force, sunlight_direction ) * sunlight_direction
        force_tangent_vec = force - force_normal_vec
        force_normal_mag[i] = np.linalg.norm(force_normal_vec)
        force_tangent_mag[i] = np.linalg.norm(force_tangent_vec)

    cached_force_lines = utils.plot_force_profile(ax2, elevation, force_mag, force_normal_mag, force_tangent_mag, sunlight_elevation, cached_force_lines)

    fig.canvas.draw()
    fig2.canvas.draw()
    fig.canvas.flush_events()
    fig2.canvas.flush_events()

def spher2cart(elevation, azimuth):
    elevation = np.pi - elevation
    x = np.sin(elevation) * np.cos(azimuth)
    y = np.sin(elevation) * np.sin(azimuth)
    z = np.cos(elevation)
    return np.array([x, y, z])

utils.setup_sliders(update_sail)

import ipywidgets as widgets
# display( widgets.HBox((fig.canvas, fig2.canvas)) )

tab = widgets.Tab()
tab.children = (fig.canvas, fig2.canvas, widgets.HBox((fig.canvas, fig2.canvas)))
tab.titles = ("3D Sail Plot", "Force Plot", "Both")
display(tab)



GridspecLayout(children=(Label(value='Sunlight Azimuth:', layout=Layout(grid_area='widget001')), FloatSlider(v…

Output()